In [1]:
!pwd

/home/techie/Desktop/general/end-to-end-liver-project/research


In [2]:
import os

os.chdir('../')
!pwd

/home/techie/Desktop/general/end-to-end-liver-project


In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/fraidoon_omarzai/end-to-end-liver-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="fraidoon_omarzai"
os.environ["MLFLOW_TRACKING_PASSWORD"]="bc25b16bd5206328d8899cf34377f26ad71d1420"

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_path: Path
    model_path: Path
    metric_file_name: str
    mlflow_uri: str
    experiment_name: str
    run_name: str
    target: str
    all_params: dict

In [5]:
from liver.constants import *
from liver.utils.common import read_yaml, create_directories, save_json

In [6]:
class ConfigurationManager:
    def __init__(self,
                 config=CONFIG_FILE_PATH,
                 params=PARAMS_FILE_PATH,
                 schema=SCHEMA_FILE_PATH):
        self.config= read_yaml(config)
        self.params= read_yaml(params)
        self.schema= read_yaml(schema)
        
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params
        schema = self.schema
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_path=config.test_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            mlflow_uri=config.mlflow_uri,
            experiment_name=config.experiment_name,
            run_name=config.run_name,
            target=schema.TARGET.name,
            all_params=self.params.Random_forest
        )
        
        return model_evaluation_config

In [7]:
from liver import logger
from sklearn.metrics import f1_score, precision_score, recall_score
import pandas as pd
import joblib
from urllib.parse import urlparse
import mlflow

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, y_true, y_pred):
        f1 = f1_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        recall = recall_score(y_true, y_pred)
        
        return f1, precision, recall
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_path)
        model = joblib.load(self.config.model_path)
        logger.info(self.config.mlflow_uri)
        test_x = test_data.drop([self.config.target], axis=1)
        test_y = test_data[[self.config.target]]
        
        mlflow.set_experiment(self.config.experiment_name)
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        
        with mlflow.start_run():

            pred = model.predict(test_x)

            (f1, precision, recall) = self.eval_metrics(test_y, pred)
            
            # Saving metrics as local
            scores = {"f1": f1, "precision": precision, "recall": recall}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("f1", f1)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="RandomForest")
            else:
                mlflow.sklearn.log_model(model, "model")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model = ModelEvaluation(model_evaluation_config)
    model.log_into_mlflow()
except Exception as e:
    raise e

[2023-10-21 08:54:41,914: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-21 08:54:41,918: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-21 08:54:41,929: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-10-21 08:54:41,934: INFO: common: created directory at: artifacts/model_evaluation]


[2023-10-21 08:54:42,234: INFO: 3954913682: https://dagshub.com/fraidoon_omarzai/end-to-end-liver-project.mlflow]


2023/10/21 08:54:54 INFO mlflow.tracking.fluent: Experiment with name 'Liver-Random-Forest' does not exist. Creating a new experiment.


[2023-10-21 08:54:59,311: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]
[2023-10-21 08:56:31,595: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2396)')': /fraidoon_omarzai/end-to-end-liver-project.mlflow/api/2.0/mlflow-artifacts/artifacts/76bb089b67a84c539d42e512e967c921/94cb53df991f453c8d0a9180dd9c2b25/artifacts/model/model.pkl]


Successfully registered model 'RandomForest'.
2023/10/21 08:57:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForest, version 1
Created version '1' of model 'RandomForest'.
